In [ ]:
import cv2
import numpy
import numpy as np

# Load the image
img = cv2.imread('chessboard.png')


# Chessboard dimensions
CHESSBOARD_CORNERS_ROWCOUNT = 9
CHESSBOARD_CORNERS_COLCOUNT = 6

# Get the perspective transform matrix
intrinsics = np.array([[1.662707630210665e+03, 0, 1.141556460466331e+03],
                       [0, 1.665176714774784e+03, 7.776969134949534e+02],
                       [0, 0, 1]])

dist = np.array([[0.106937435542469, -0.193726569015277, 0, 0, 0]])

h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(intrinsics, dist, (w,h), 1, (w,h))

img = cv2.undistort(img, intrinsics, dist)
# crop the image

x, y, w, h = roi
img = img[y:y+h, x:x+w]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (CHESSBOARD_CORNERS_ROWCOUNT, CHESSBOARD_CORNERS_COLCOUNT), None)
corners_int = corners.astype(numpy.int8)
# Bottom right corner has pixel value (90, 60), Top left has (0, 0), cornerns_new is a grid of 54 points
corners_new = np.array([[x, y] for y in range(corners_int[-1][0][1], corners_int[-1][0][1]+60, 10) for x in range(corners_int[-1][0][0], corners_int[-1][0][0]+90, 10)])
print(corners_new)
if ret:
    cv2.drawChessboardCorners(img, (CHESSBOARD_CORNERS_ROWCOUNT, CHESSBOARD_CORNERS_COLCOUNT), corners, ret)


    H, _ = cv2.findHomography(corners, corners_new)
    print(H)

    img = cv2.warpPerspective(img, H, (img.shape[1], img.shape[0]))
    img = cv2.resize(img, (1280, 720))


    # Display the result
    cv2.imshow('Bird\'s Eye View', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Chessboard corners not found.")